### Import statements

Since Great Expectations is listed in the requirements.txt file, Deepnote will install it when the hardware starts. Read more about package installation here.

In [ ]:
# CS 6603 Final Group project: Mark P Abbott - mabbott7, Michael Countouris - mcountouris3, Soon Ryu - sryu71

# Import libraries we need for this project
import pandas as pd  # for working with data tables
import numpy as np  # for math calculations
import matplotlib.pyplot as plt  # for making graphs
import seaborn as sns  # for making nice graphs
from sklearn.preprocessing import RobustScaler  # for scaling data
from sklearn.model_selection import train_test_split  # for splitting data into train/test
from sklearn import tree  # for decision tree model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # for checking model performance
from aif360.datasets import BinaryLabelDataset  # for fairness analysis
from aif360.algorithms.preprocessing import Reweighing  # for fixing bias in data

In [ ]:
# STEP 1: Dataset Analysis

# Load the CSV file into a data table
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv', encoding='utf-8-sig', on_bad_lines='skip')

print("="*60)
print("STEP 1: DATASET ANALYSIS RESULTS")
print("="*60)

# Show which dataset we are using
print("\n1. DATASET SELECTED: Employee Attrition")
print("   Name: WA_Fn-UseC_-HR-Employee-Attrition.csv")
print("   Link: https://www.kaggle.com/datasets/patelprashant/employee-attrition")

# Show what domain this is
print("\n2. REGULATED DOMAIN:")
print("   Employment (HR/Human Resources)")

# Count how many rows in the data
print(f"\n3. NUMBER OF OBSERVATIONS:")
print(f"   {len(df)} observations")

# Count how many columns in the data
print(f"\n4. NUMBER OF VARIABLES:")
print(f"   {len(df.columns)} variables")

# Show the outcome variables we will predict
print(f"\n5. DEPENDENT/OUTCOME VARIABLES:")
print("   - Attrition (Primary): Employee turnover (Yes/No)")
print(f"     Values: {list(df['Attrition'].unique())}")

# Count how many Yes and No for Attrition
attrition_counts = df['Attrition'].value_counts()
total = len(df)
no_pct = (attrition_counts['No'] / total) * 100
yes_pct = (attrition_counts['Yes'] / total) * 100
print(f"     Distribution: No: {attrition_counts['No']} ({no_pct:.1f}%), Yes: {attrition_counts['Yes']} ({yes_pct:.1f}%)")

print()
print("   - PerformanceRating (Secondary): Performance score")
print(f"     Values: {list(df['PerformanceRating'].unique())}")

# Count performance ratings
perf_counts = df['PerformanceRating'].value_counts()
rating3_pct = (perf_counts[3] / total) * 100
rating4_pct = (perf_counts[4] / total) * 100
print(f"     Distribution: Rating 3: {perf_counts[3]} ({rating3_pct:.1f}%), Rating 4: {perf_counts[4]} ({rating4_pct:.1f}%)")

# Show protected class variables (Age, Gender, MaritalStatus)
print(f"\n6. PROTECTED CLASS VARIABLES:")
print(f"   Total: 3 protected class variables")
print()
print("   - Age (Continuous)")
print(f"     Range: {df['Age'].min()} to {df['Age'].max()} years")
print()
print("   - Gender (Categorical)")
print(f"     Values: {list(df['Gender'].unique())}")

# Count male and female
gender_counts = df['Gender'].value_counts()
male_pct = (gender_counts['Male'] / total) * 100
female_pct = (gender_counts['Female'] / total) * 100
print(f"     Distribution: Male: {gender_counts['Male']} ({male_pct:.1f}%), Female: {gender_counts['Female']} ({female_pct:.1f}%)")

print()
print("   - MaritalStatus (Categorical)")
print(f"     Values: {list(df['MaritalStatus'].unique())}")

# Count married, single, divorced
marital_counts = df['MaritalStatus'].value_counts()
married_pct = (marital_counts['Married'] / total) * 100
single_pct = (marital_counts['Single'] / total) * 100
divorced_pct = (marital_counts['Divorced'] / total) * 100
print(f"     Distribution: Married: {marital_counts['Married']} ({married_pct:.1f}%), Single: {marital_counts['Single']} ({single_pct:.1f}%), Divorced: {marital_counts['Divorced']} ({divorced_pct:.1f}%)")

# Show the laws that protect each class
print(f"\n7. LEGAL PRECEDENCE FOR EACH PROTECTED CLASS:")
print("   - Age: Age Discrimination in Employment Act (ADEA)")
print("         Protects workers 40+ years old")
print()
print("   - Gender: Title VII of Civil Rights Act of 1964")
print("           Prohibits employment discrimination based on sex")
print()
print("   - MaritalStatus: Fair Employment and Housing Act")
print("                  Prohibits discrimination based on marital status")

# Check if data meets project requirements
print(f"\n8. PROJECT REQUIREMENTS CHECK:")
print(f"   ✓ At least 500 observations? YES ({len(df)} observations)")
print(f"   ✓ At least 2 protected classes? YES (3 classes)")
print(f"   ✓ At least 2 dependent variables? YES (2 variables)")
print(f"   ✓ Related to regulated domain? YES (Employment)")

# Check for missing data
print(f"\n9. DATA QUALITY CHECK:")
missing_count = df.isnull().sum().sum()
if missing_count == 0:
    print("   ✓ No missing values found!")
else:
    print(f"   ⚠ {missing_count} missing values found")

In [ ]:
# Check if AIF360 library is installed
# AIF360 is used for fairness analysis
try:
    import aif360
    print("✅ AIF360 successfully installed!")
    print("Version:", aif360.__version__)
except ImportError as e:
    print("❌ AIF360 installation failed:", e)

In [ ]:
# Step 2: Dataset exploration

# 2.1 Show all the protected class groups
print("=== STEP 2.1: Protected Class Subgroups ===")
print("Age groups:", df['Age'].unique()[:10], "... (continuous)")
print("Gender:", df['Gender'].unique())
print("MaritalStatus:", df['MaritalStatus'].unique())

# 2.2 Convert text to numbers
print("\n=== STEP 2.2: Discretize Subgroups ===")
# Change Male/Female to 0/1
df['Gender_num'] = df['Gender'].map({'Male': 0, 'Female': 1})
# Change marital status to numbers
df['MaritalStatus_num'] = df['MaritalStatus'].map({'Single': 0, 'Married': 1, 'Divorced': 2})
# Change Attrition to numbers
df['Attrition_num'] = df['Attrition'].map({'No': 0, 'Yes': 1})
# Change performance rating to numbers
df['PerformanceRating_num'] = df['PerformanceRating'].map({3: 0, 4: 1})

print("Gender mapping: Male=0, Female=1")
print("MaritalStatus mapping: Single=0, Married=1, Divorced=2") 
print("Attrition mapping: No=0, Yes=1")
print("PerformanceRating mapping: 3=0, 4=1")

# 2.3 Choose two protected classes for analysis
print("\n=== STEP 2.3: Selected Protected Classes ===")
print("1. Gender")
print("2. MaritalStatus")

# 2.4 Make tables to count combinations
print("\n=== STEP 2.4: Frequency Tables ===")
print("Gender vs Attrition:")
print(pd.crosstab(df['Gender'], df['Attrition']))

print("\nMaritalStatus vs Attrition:")
print(pd.crosstab(df['MaritalStatus'], df['Attrition']))

print("\nGender vs PerformanceRating:")
print(pd.crosstab(df['Gender'], df['PerformanceRating']))

print("\nMaritalStatus vs PerformanceRating:")
print(pd.crosstab(df['MaritalStatus'], df['PerformanceRating']))

In [ ]:
# Step 2.5: Create bar charts to visualize the data
plt.figure(figsize=(15, 10))

# Chart 1: Count of Gender vs Attrition
plt.subplot(2, 2, 1)
gender_attrition = pd.crosstab(df['Gender'], df['Attrition'])
gender_attrition.plot(kind='bar', ax=plt.gca())
plt.title('Gender vs Attrition')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=45)

# Chart 2: Count of MaritalStatus vs Attrition  
plt.subplot(2, 2, 2)
marital_attrition = pd.crosstab(df['MaritalStatus'], df['Attrition'])
marital_attrition.plot(kind='bar', ax=plt.gca())
plt.title('Marital Status vs Attrition')
plt.xlabel('Marital Status')
plt.ylabel('Count')
plt.xticks(rotation=45)

# Chart 3: Count of Gender vs PerformanceRating
plt.subplot(2, 2, 3)
gender_performance = pd.crosstab(df['Gender'], df['PerformanceRating'])
gender_performance.plot(kind='bar', ax=plt.gca())
plt.title('Gender vs Performance Rating')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=45)

# Chart 4: Count of MaritalStatus vs PerformanceRating
plt.subplot(2, 2, 4)
marital_performance = pd.crosstab(df['MaritalStatus'], df['PerformanceRating'])
marital_performance.plot(kind='bar', ax=plt.gca())
plt.title('Marital Status vs Performance Rating')
plt.xlabel('Marital Status')
plt.ylabel('Count')
plt.xticks(rotation=45)

# Save and show the charts
plt.tight_layout()
plt.savefig('2_5_protected_class_charts.png')
plt.show()
plt.close()

In [ ]:
# Data Preprocessing: Prepare data for machine learning model

# Convert OverTime to numbers (1 = No, 0 = Yes)
df['OverTime'] = (df['OverTime'] == 'No').astype(int)

# These columns don't help - remove them
drop_cols = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']

# These columns have categories like job role, department
categorical_features = ['BusinessTravel', 'Department', 'EducationField', 'JobRole']

# These columns have numbers like salary, age
numerical_features = ['DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'RelationshipSatisfaction', 'StockOptionLevel',
 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

# These columns we will convert to binary later
variable_features = ['Age', 'Gender', 'MaritalStatus', 'PerformanceRating', 'Attrition'] 

# Remove useless columns
df = df.drop(columns=drop_cols)

# Get categorical columns
df_cat = df[categorical_features]

# Convert categories to binary columns (OneHotEncoding)
df_ohe = pd.get_dummies(df_cat, dtype=int)
# Add back numerical columns
df_ohe[numerical_features] = df[numerical_features]
# Add back variable columns
df_ohe[variable_features] = df[variable_features]
# Replace old dataframe with new one
df = df_ohe.copy()

In [ ]:
print("3.1: PRIVILEGED AND UNPRIVILEGED GROUPS:\n")

# Define which groups are privileged and unprivileged for each protected class
protected_classes_info = {
    'Age': {
        'privileged': 'Less than 40 years old',  # younger workers
        'unprivileged': '40+ years old'  # older workers
    },
    'Gender': {
        'privileged': 'Male',  # men
        'unprivileged': 'Female'  # women
    },
    'MaritalStatus': {
        'privileged': 'Married',  # married people
        'unprivileged': 'Single or Divorced'  # not married
    }
}

# Print the information
for pc, groups in protected_classes_info.items():
    print(f"Protected Class: {pc}")
    print(f"- Privileged Group: {groups['privileged']}")
    print(f"- Unprivileged Group: {groups['unprivileged']}")

In [ ]:
print("Creating binary variable mappings")

# Convert protected classes to binary (0 or 1)
# 1 = privileged group, 0 = unprivileged group

df['Age_Bin'] = (df['Age'] < 40).astype(int)  # 1 = younger than 40, 0 = 40 or older
df['Gender_Bin'] = (df['Gender'] == 'Male').astype(int)  # 1 = Male, 0 = Female
df['MaritalStatus_Bin'] = (df['MaritalStatus'] == 'Married').astype(int)  # 1 = Married, 0 = Single or Divorced

# Convert outcomes to binary
df['Attrition_Bin'] = (df['Attrition'] == 'No').astype(int)  # 1 = stayed, 0 = left
df['PerformanceRating_Bin'] = (df['PerformanceRating'] == 4).astype(int)  # 1 = high rating, 0 = low rating

In [ ]:
print("Fairness Metric Functions")

# Function 1: Disparate Impact
# Measures fairness by comparing rates between groups
# Goal: value should be close to 1.0 (fair is 0.8 to 1.25)
def compute_disparate_impact(df, protected_attr, outcome_attr):
    # Split into privileged (1) and unprivileged (0) groups
    priv = df[df[protected_attr] == 1]
    unpriv = df[df[protected_attr] == 0]

    # Check if groups exist
    if len(priv) == 0 or len(unpriv) == 0:
        return np.nan

    # Calculate positive outcome rate for each group
    priv_rate = priv[outcome_attr].sum() / len(priv)
    unpriv_rate = unpriv[outcome_attr].sum() / len(unpriv)

    # Can't divide by zero
    if priv_rate == 0:
        return np.nan
    # Return ratio of rates
    return unpriv_rate / priv_rate

# Function 2: Statistical Parity Difference
# Measures fairness by comparing differences between groups
# Goal: value should be close to 0.0 (fair is -0.1 to 0.1)
def compute_statistical_parity_difference(df, protected_attr, outcome_attr):
    # Split into privileged (1) and unprivileged (0) groups
    priv = df[df[protected_attr] == 1]
    unpriv = df[df[protected_attr] == 0]

    # Check if groups exist
    if len(priv) == 0 or len(unpriv) == 0:
        return np.nan

    # Calculate positive outcome rate for each group
    priv_rate = priv[outcome_attr].sum() / len(priv)
    unpriv_rate = unpriv[outcome_attr].sum() / len(unpriv)
    # Return difference of rates
    return unpriv_rate - priv_rate

In [ ]:
print("3.2: COMPUTE FAIRNESS METRICS - ORIGINAL DATASET")

# Calculate fairness for Gender -> Attrition
di_gender_attr = compute_disparate_impact(df, 'Gender_Bin', 'Attrition_Bin')
spd_gender_attr = compute_statistical_parity_difference(df, 'Gender_Bin', 'Attrition_Bin')

# Calculate fairness for Gender -> PerformanceRating
di_gender_perf = compute_disparate_impact(df, 'Gender_Bin', 'PerformanceRating_Bin')
spd_gender_perf = compute_statistical_parity_difference(df, 'Gender_Bin', 'PerformanceRating_Bin')

# Calculate fairness for MaritalStatus -> Attrition
di_maritalStatus_attr = compute_disparate_impact(df, 'MaritalStatus_Bin', 'Attrition_Bin')
spd_maritalStatus_attr = compute_statistical_parity_difference(df, 'MaritalStatus_Bin', 'Attrition_Bin')

# Calculate fairness for MaritalStatus -> PerformanceRating
di_maritalStatus_perf = compute_disparate_impact(df, 'MaritalStatus_Bin', 'PerformanceRating_Bin')
spd_maritalStatus_perf = compute_statistical_parity_difference(df, 'MaritalStatus_Bin', 'PerformanceRating_Bin')

# Put all results in a table
results_original = pd.DataFrame({
    'Protected Class': ['Gender', 'Gender', 'MaritalStatus', 'MaritalStatus'],
    'Outcome Variable': ['Attrition', 'PerformanceRating', 'Attrition', 'PerformanceRating'],
    'Disparate Impact': [di_gender_attr, di_gender_perf, di_maritalStatus_attr, di_maritalStatus_perf],
    'Statistical Parity Difference': [spd_gender_attr, spd_gender_perf, spd_maritalStatus_attr, spd_maritalStatus_perf]
})

# Show the table
print()
print("SUMMARY TABLE - ORIGINAL DATASET:")
print(results_original.to_string(index=False))

In [ ]:
print("3.3: APPLY REWEIGHING (AIF360)")

# Get only number columns
df_numeric = df.select_dtypes(include=[np.number]).copy()

# Create AIF360 dataset format
dataset_orig = BinaryLabelDataset(
    favorable_label=1,  # 1 = good outcome
    unfavorable_label=0,  # 0 = bad outcome
    df=df_numeric,
    label_names=['Attrition_Bin'],  # what we want to predict
    protected_attribute_names=['Gender_Bin', 'MaritalStatus_Bin']  # protected classes
)

# Define privileged group (Male and Married)
priv_groups = [{'Gender_Bin': 1, 'MaritalStatus_Bin': 1}]
# Define unprivileged groups (all other combinations)
unpriv_groups = [
    {'Gender_Bin': 0, 'MaritalStatus_Bin': 0},  # Female and Single/Divorced
    {'Gender_Bin': 0, 'MaritalStatus_Bin': 1},  # Female and Married
    {'Gender_Bin': 1, 'MaritalStatus_Bin': 0}   # Male and Single/Divorced
]

# Create reweighing object
RW = Reweighing(privileged_groups=priv_groups,unprivileged_groups=unpriv_groups)

# Apply reweighing to fix bias
dataset_transf = RW.fit_transform(dataset_orig)

# Add weights back to dataframe
df.loc[df_numeric.index, 'weight'] = dataset_transf.instance_weights

# Show weight range
print(f"Weight range: [{df['weight'].min()}, {df['weight'].max()}]")

In [ ]:
print("Weighted Fairness Metric Functions")

# Helper function: Calculate weighted rate
# This accounts for the weights we added with reweighing
def _weighted_rate(group_df, outcome_attr):
    w = group_df['weight']  # get weights
    if w.sum() == 0:  # avoid divide by zero
        return np.nan
    # Calculate weighted average
    return (group_df[outcome_attr] * w).sum() / w.sum()

# Disparate Impact with weights
def compute_disparate_impact_weighted(df, protected_attr, outcome_attr):
    # Split into privileged and unprivileged groups
    priv = df[df[protected_attr] == 1]
    unpriv = df[df[protected_attr] == 0]
    # Get weighted rates for each group
    priv_rate = _weighted_rate(priv, outcome_attr)
    unpriv_rate = _weighted_rate(unpriv, outcome_attr)
    # Can't divide by zero
    if priv_rate == 0:
        return np.nan
    # Return ratio
    return unpriv_rate / priv_rate

# Statistical Parity Difference with weights
def compute_statistical_parity_difference_weighted(df, protected_attr, outcome_attr):
    # Split into privileged and unprivileged groups
    priv = df[df[protected_attr] == 1]
    unpriv = df[df[protected_attr] == 0]
    # Get weighted rates for each group
    priv_rate = _weighted_rate(priv, outcome_attr)
    unpriv_rate = _weighted_rate(unpriv, outcome_attr)
    # Return difference
    return unpriv_rate - priv_rate

In [ ]:
print("3.4: COMPUTE FAIRNESS METRICS - TRANSFORMED DATASET")

# Calculate weighted fairness for Gender -> Attrition
di_gender_attr_w = compute_disparate_impact_weighted(df, 'Gender_Bin', 'Attrition_Bin')
spd_gender_attr_w = compute_statistical_parity_difference_weighted(df, 'Gender_Bin', 'Attrition_Bin')

# Calculate weighted fairness for Gender -> PerformanceRating
di_gender_perf_w = compute_disparate_impact_weighted(df, 'Gender_Bin', 'PerformanceRating_Bin')
spd_gender_perf_w = compute_statistical_parity_difference_weighted(df, 'Gender_Bin', 'PerformanceRating_Bin')

# Calculate weighted fairness for MaritalStatus -> Attrition
di_maritalStatus_attr_w = compute_disparate_impact_weighted(df, 'MaritalStatus_Bin', 'Attrition_Bin')
spd_maritalStatus_attr_w = compute_statistical_parity_difference_weighted(df, 'MaritalStatus_Bin', 'Attrition_Bin')

# Calculate weighted fairness for MaritalStatus -> PerformanceRating
di_maritalStatus_perf_w = compute_disparate_impact_weighted(df, 'MaritalStatus_Bin', 'PerformanceRating_Bin')
spd_maritalStatus_perf_w = compute_statistical_parity_difference_weighted(df, 'MaritalStatus_Bin', 'PerformanceRating_Bin')

# Put all results in a table
results_transformed = pd.DataFrame({
    'Protected Class': ['Gender', 'Gender', 'MaritalStatus', 'MaritalStatus'],
    'Outcome Variable': ['Attrition', 'PerformanceRating', 'Attrition', 'PerformanceRating'],
    'Disparate Impact': [di_gender_attr_w, di_gender_perf_w, di_maritalStatus_attr_w, di_maritalStatus_perf_w],
    'Statistical Parity Difference': [spd_gender_attr_w, spd_gender_perf_w, spd_maritalStatus_attr_w, spd_maritalStatus_perf_w]
})

# Show the table
print()
print("SUMMARY TABLE - TRANSFORMED DATASET:")
print(results_transformed.to_string(index=False))

In [ ]:
# Step 4 Setup: Create two datasets for comparison

# Original dataset: no changes (weight = 1.0 for all rows)
df_original = df.copy()
df_original['weight'] = 1.0

# Transformed dataset: has weights from reweighing
df_transformed = df.copy()

# Get the target labels (what we want to predict)
y_original = df_original['Attrition_Bin']  # 1 = stayed, 0 = left
y_transformed = df_transformed['Attrition_Bin']

# Get the weights for transformed dataset
sample_weights = df_transformed['weight']

print("Step 4 handoff:")
print("- df_original")
print("- df_transformed")
print("- y_original")
print("- y_transformed")
print("- sample_weights")

In [ ]:
print("Steps 4.1, 4.2, 4.4, 4.5- Splitting data and training models")

# Remove columns we don't need anymore
additions = ['Attrition_Bin', 'weight']
step_3_drop = variable_features + additions
df_original = df_original.drop(columns=step_3_drop)
df_transformed = df_transformed.drop(columns=step_3_drop)

# Scale numerical features to similar range
# RobustScaler works well with outliers
scaler = RobustScaler()
df_original[numerical_features] = scaler.fit_transform(df_original[numerical_features])
df_transformed[numerical_features] = scaler.fit_transform(df_transformed[numerical_features])

# Split original data into train (80%) and test (20%)
x_train_org, x_test_org, y_train_org, y_test_org = train_test_split(
    df_original, y_original, test_size=.2, random_state=42)

# Split transformed data into train and test
x_train_trans, x_test_trans, y_train_trans, y_test_trans, weights_train, weights_test = train_test_split(
    df_transformed, y_transformed, sample_weights, test_size=.2, random_state=42)

# Train Decision Tree model on original data
# Decision Tree learns rules to predict if employee will leave
original_tree = tree.DecisionTreeClassifier(max_depth=5, random_state=42)
original_tree.fit(x_train_org, y_train_org)  # learn from training data
org_pred = original_tree.predict(x_train_org)  # make predictions on training data
org_test_pred = original_tree.predict(x_test_org)  # make predictions on test data
df_pred = x_train_org.copy()  # create new dataframe
df_pred['prediction'] = org_pred  # add predictions
print("Original Tree Metrics:")
print("Training Accuracy:", accuracy_score(y_train_org,org_pred))
print("Test Set Accuracy: ", accuracy_score(y_test_org, org_test_pred))

# Train Decision Tree model on transformed data (with weights)
transformed_tree = tree.DecisionTreeClassifier(max_depth=5, random_state=42)
transformed_tree.fit(x_train_trans, y_train_trans, sample_weight=weights_train)  # use weights to reduce bias
trans_pred = transformed_tree.predict(x_train_trans)  # make predictions on training data
trans_test_pred = transformed_tree.predict(x_test_org)  # make predictions on test data
df_pred_transformed = x_train_trans.copy()  # create new dataframe
df_pred_transformed['weight'] = weights_train.copy()  # keep weights for fairness calculations
df_pred_transformed['prediction'] = trans_pred  # add predictions
print()
print("Transformed Tree Metrics:")
print("Training Accuracy:", accuracy_score(y_train_trans,trans_pred))
print("Test Set Accuracy: ", accuracy_score(y_test_trans, trans_test_pred))

In [ ]:
print("Steps 4.3 and 4.6- Calculating fairness on model predictions")

# Calculate fairness metrics for ORIGINAL model predictions
# Gender fairness
di_gender_pred = compute_disparate_impact(df_pred, 'Gender_Bin', 'prediction')
spd_gender_pred = compute_statistical_parity_difference(df_pred, 'Gender_Bin', 'prediction')

# MaritalStatus fairness
di_maritalStatus_pred = compute_disparate_impact(df_pred, 'MaritalStatus_Bin', 'prediction')
spd_maritalStatus_pred = compute_statistical_parity_difference(df_pred, 'MaritalStatus_Bin', 'prediction')

# Show results in table
results_original_pred = pd.DataFrame({
    'Protected Class': ['Gender', 'MaritalStatus'],
    'Outcome Variable': ['Attrition', 'Attrition'],
    'Disparate Impact': [di_gender_pred, di_maritalStatus_pred],
    'Statistical Parity Difference': [spd_gender_pred, spd_maritalStatus_pred]
})

print()
print("SUMMARY TABLE - ORIGINAL DATASET:")
print(results_original_pred.to_string(index=False))


# Calculate fairness metrics for TRANSFORMED model predictions (with weights)
# Gender fairness
di_gender_attr_w_pred = compute_disparate_impact_weighted(df_pred_transformed, 'Gender_Bin', 'prediction')
spd_gender_attr_w_pred = compute_statistical_parity_difference_weighted(df_pred_transformed, 'Gender_Bin', 'prediction')

# MaritalStatus fairness
di_maritalStatus_attr_w = compute_disparate_impact_weighted(df_pred_transformed, 'MaritalStatus_Bin', 'prediction')
spd_maritalStatus_attr_w = compute_statistical_parity_difference_weighted(df_pred_transformed, 'MaritalStatus_Bin', 'prediction')

# Show results in table
results_transformed_pred = pd.DataFrame({
    'Protected Class': ['Gender', 'MaritalStatus'],
    'Outcome Variable': ['Attrition', 'Attrition'],
    'Disparate Impact': [di_gender_attr_w, di_maritalStatus_attr_w],
    'Statistical Parity Difference': [spd_gender_attr_w, spd_maritalStatus_attr_w]
})

print()
print("SUMMARY TABLE - TRANSFORMED DATASET:")
print(results_transformed_pred.to_string(index=False))

In [ ]:
print("="*80)
print("STEP 5: ANALYSIS")
print("="*80)

print("\n" + "="*80)
print("5.1: FAIRNESS METRICS COMPARISON")
print("="*80)

# Collect all fairness metrics from all stages
# Stage 1: Original Dataset (Step 3.2)
# Stage 2: Transformed Dataset (Step 3.4)  
# Stage 3: Classifier Predictions (Steps 4.3 and 4.6)

# Get Gender fairness values from all stages
gender_di_original_data = di_gender_attr
gender_di_transformed_data = di_gender_attr_w
gender_di_original_classifier = di_gender_pred
gender_di_transformed_classifier = di_gender_attr_w_pred

gender_spd_original_data = spd_gender_attr
gender_spd_transformed_data = spd_gender_attr_w
gender_spd_original_classifier = spd_gender_pred
gender_spd_transformed_classifier = spd_gender_attr_w_pred

# Get MaritalStatus fairness values from all stages
marital_di_original_data = di_maritalStatus_attr
marital_di_transformed_data = di_maritalStatus_attr_w
marital_di_original_classifier = di_maritalStatus_pred
marital_di_transformed_classifier = di_maritalStatus_attr_w

marital_spd_original_data = spd_maritalStatus_attr
marital_spd_transformed_data = spd_maritalStatus_attr_w
marital_spd_original_classifier = spd_maritalStatus_pred
marital_spd_transformed_classifier = spd_maritalStatus_attr_w

# Create comparison table
comparison_data = {
    'Stage': ['Original Data', 'Transformed Data', 'Original Classifier', 'Transformed Classifier'],
    'Gender - Disparate Impact': [gender_di_original_data, gender_di_transformed_data, 
                  gender_di_original_classifier, gender_di_transformed_classifier],
    'Gender - Statistical Parity Difference': [gender_spd_original_data, gender_spd_transformed_data, 
                  gender_spd_original_classifier, gender_spd_transformed_classifier],
    'MaritalStatus - Disparate Impact': [marital_di_original_data, marital_di_transformed_data, 
                        marital_di_original_classifier, marital_di_transformed_classifier],
    'MaritalStatus - Statistical Parity Difference': [marital_spd_original_data, marital_spd_transformed_data, 
                         marital_spd_original_classifier, marital_spd_transformed_classifier]
}

comparison_df = pd.DataFrame(comparison_data)

print("\nFAIRNESS METRICS COMPARISON TABLE:")
print(comparison_df.to_string(index=False))

# Create graphs to show results
fig = plt.figure(figsize=(20, 12))
stages = ['Original\nData', 'Transformed\nData', 'Original\nClassifier', 'Transformed\nClassifier']
x_pos = np.arange(len(stages))
bar_width = 0.35

# Graph 1: Gender Disparate Impact
ax1 = plt.subplot(2, 2, 1)
bars = ax1.bar(x_pos, comparison_df['Gender - Disparate Impact'], bar_width, 
               color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'], alpha=0.8)
ax1.axhline(1.0, color='black', linestyle='--', linewidth=2, label='Ideal Fairness (DI=1.0)')
ax1.axhspan(0.8, 1.25, color='green', alpha=0.15, label='Fair Region (0.8-1.25)')
ax1.set_ylabel('Disparate Impact', fontsize=12, fontweight='bold')
ax1.set_title('Gender - Disparate Impact', fontsize=14, fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(stages, fontsize=10)
ax1.set_ylim(0.7, 1.4)
ax1.legend(loc='upper right')
ax1.grid(axis='y', alpha=0.3)
for i, (bar, val) in enumerate(zip(bars, comparison_df['Gender - Disparate Impact'])):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{val:.3f}', ha='center', va='bottom', fontweight='bold')

# Graph 2: Gender Statistical Parity Difference
ax2 = plt.subplot(2, 2, 2)
bars = ax2.bar(x_pos, comparison_df['Gender - Statistical Parity Difference'], bar_width, 
              color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'], alpha=0.8)
ax2.axhline(0.0, color='black', linestyle='--', linewidth=2, label='Ideal Fairness (SPD=0.0)')
ax2.axhspan(-0.1, 0.1, color='green', alpha=0.15, label='Fair Region (-0.1 to 0.1)')
ax2.set_ylabel('Statistical Parity Difference', fontsize=12, fontweight='bold')
ax2.set_title('Gender - Statistical Parity Difference', fontsize=14, fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(stages, fontsize=10)
ax2.set_ylim(-0.1, 0.1)
ax2.legend(loc='upper right')
ax2.grid(axis='y', alpha=0.3)
for i, (bar, val) in enumerate(zip(bars, comparison_df['Gender - Statistical Parity Difference'])):
    ax2.text(bar.get_x() + bar.get_width()/2, 
             bar.get_height() + (0.005 if val >= 0 else -0.01), 
            f'{val:.3f}', ha='center', va='bottom' if val >= 0 else 'top', fontweight='bold')

# Graph 3: MaritalStatus Disparate Impact
ax3 = plt.subplot(2, 2, 3)
bars = ax3.bar(x_pos, comparison_df['MaritalStatus - Disparate Impact'], bar_width, 
              color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'], alpha=0.8)
ax3.axhline(1.0, color='black', linestyle='--', linewidth=2, label='Ideal Fairness (DI=1.0)')
ax3.axhspan(0.8, 1.25, color='green', alpha=0.15, label='Fair Region (0.8-1.25)')
ax3.set_ylabel('Disparate Impact', fontsize=12, fontweight='bold')
ax3.set_title('MaritalStatus - Disparate Impact', fontsize=14, fontweight='bold')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(stages, fontsize=10)
ax3.set_ylim(0.7, 1.4)
ax3.legend(loc='upper right')
ax3.grid(axis='y', alpha=0.3)
for i, (bar, val) in enumerate(zip(bars, comparison_df['MaritalStatus - Disparate Impact'])):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{val:.3f}', ha='center', va='bottom', fontweight='bold')

# Graph 4: MaritalStatus Statistical Parity Difference
ax4 = plt.subplot(2, 2, 4)
bars = ax4.bar(x_pos, comparison_df['MaritalStatus - Statistical Parity Difference'], bar_width, 
              color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'], alpha=0.8)
ax4.axhline(0.0, color='black', linestyle='--', linewidth=2, label='Ideal Fairness (SPD=0.0)')
ax4.axhspan(-0.1, 0.1, color='green', alpha=0.15, label='Fair Region (-0.1 to 0.1)')
ax4.set_ylabel('Statistical Parity Difference', fontsize=12, fontweight='bold')
ax4.set_title('MaritalStatus - Statistical Parity Difference', fontsize=14, fontweight='bold')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(stages, fontsize=10)
ax4.set_ylim(-0.15, 0.05)
ax4.legend(loc='upper right')
ax4.grid(axis='y', alpha=0.3)
for i, (bar, val) in enumerate(zip(bars, comparison_df['MaritalStatus - Statistical Parity Difference'])):
    ax4.text(bar.get_x() + bar.get_width()/2, 
            bar.get_height() + (0.005 if val >= 0 else -0.01), 
            f'{val:.3f}', ha='center', va='bottom' if val >= 0 else 'top', fontweight='bold')

plt.suptitle('Fairness Metrics Analysis Across All Stages\n(Employee Attrition Prediction)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.savefig('Step_5_Fairness_Analysis.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print("\n" + "="*80)
print("Graph saved as: Step_5_Fairness_Analysis.png")
print("="*80)

In [ ]:
print("\n" + "="*80)
print("5.2: CHANGE ANALYSIS")
print("="*80)

# Function to check if fairness got better, worse, or stayed the same
def classify_change(original, transformed, metric_type):
    # Small changes below this number are ignored
    threshold = 0.005
    
    if metric_type == 'DI':
        # For Disparate Impact: goal is to get close to 1.0
        original_distance = abs(original - 1.0)  # how far from 1.0
        transformed_distance = abs(transformed - 1.0)  # how far from 1.0
        
        # Check if change is big enough
        if abs(transformed_distance - original_distance) < threshold:
            return 'No Change'
        elif transformed_distance < original_distance:
            return 'Positive Change (↑ Fairer)'  # got closer to 1.0
        else:
            return 'Negative Change (↓ Less Fair)'  # got farther from 1.0
    else:  # SPD
        # For Statistical Parity Difference: goal is to get close to 0.0
        original_distance = abs(original)  # how far from 0.0
        transformed_distance = abs(transformed)  # how far from 0.0
        
        # Check if change is big enough
        if abs(transformed_distance - original_distance) < threshold:
            return 'No Change'
        elif transformed_distance < original_distance:
            return 'Positive Change (↑ Fairer)'  # got closer to 0.0
        else:
            return 'Negative Change (↓ Less Fair)'  # got farther from 0.0

# TABLE 1: Show how fairness changed after we fixed the data
print("\nTABLE 1: CHANGES AFTER DATA TRANSFORMATION (Reweighing)")
print("-" * 80)

# Collect fairness values before and after transformation
transform_changes = {
    'Protected Class': ['Gender', 'Gender', 'MaritalStatus', 'MaritalStatus'],
    'Fairness Metric': ['Disparate Impact', 'Statistical Parity Diff', 'Disparate Impact', 'Statistical Parity Diff'],
    'Original Value': [gender_di_original_data, gender_spd_original_data, 
                      marital_di_original_data, marital_spd_original_data],
    'Transformed Value': [gender_di_transformed_data, gender_spd_transformed_data,
                         marital_di_transformed_data, marital_spd_transformed_data],
    'Change Direction': [
        classify_change(gender_di_original_data, gender_di_transformed_data, 'DI'),
        classify_change(gender_spd_original_data, gender_spd_transformed_data, 'SPD'),
        classify_change(marital_di_original_data, marital_di_transformed_data, 'DI'),
        classify_change(marital_spd_original_data, marital_spd_transformed_data, 'SPD')
    ]
}

# Make table and show results
df_transform_changes = pd.DataFrame(transform_changes)
df_transform_changes['Absolute Change'] = abs(df_transform_changes['Transformed Value'] - 
                                               df_transform_changes['Original Value'])
print(df_transform_changes.to_string(index=False))

# TABLE 2: Show how fairness changed after training the model
print("\n" + "="*80)
print("TABLE 2: CHANGES AFTER CLASSIFIER TRAINING")
print("-" * 80)
print("\nA) Original Dataset - Data vs Classifier Predictions")
print("-" * 80)

# Compare original data to original model predictions
classifier_original_changes = {
    'Protected Class': ['Gender', 'Gender', 'MaritalStatus', 'MaritalStatus'],
    'Fairness Metric': ['Disparate Impact', 'Statistical Parity Diff', 'Disparate Impact', 'Statistical Parity Diff'],
    'Data Value': [gender_di_original_data, gender_spd_original_data,
                   marital_di_original_data, marital_spd_original_data],
    'Classifier Value': [gender_di_original_classifier, gender_spd_original_classifier,
                        marital_di_original_classifier, marital_spd_original_classifier],
    'Change Direction': [
        classify_change(gender_di_original_data, gender_di_original_classifier, 'DI'),
        classify_change(gender_spd_original_data, gender_spd_original_classifier, 'SPD'),
        classify_change(marital_di_original_data, marital_di_original_classifier, 'DI'),
        classify_change(marital_spd_original_data, marital_spd_original_classifier, 'SPD')
    ]
}

# Make table
df_classifier_orig = pd.DataFrame(classifier_original_changes)
df_classifier_orig['Absolute Change'] = abs(df_classifier_orig['Classifier Value'] - 
                                            df_classifier_orig['Data Value'])
print(df_classifier_orig.to_string(index=False))

print("\n" + "-" * 80)
print("B) Transformed Dataset - Data vs Classifier Predictions")
print("-" * 80)

# Compare transformed data to transformed model predictions
classifier_transform_changes = {
    'Protected Class': ['Gender', 'Gender', 'MaritalStatus', 'MaritalStatus'],
    'Fairness Metric': ['Disparate Impact', 'Statistical Parity Diff', 'Disparate Impact', 'Statistical Parity Diff'],
    'Data Value': [gender_di_transformed_data, gender_spd_transformed_data,
                   marital_di_transformed_data, marital_spd_transformed_data],
    'Classifier Value': [gender_di_transformed_classifier, gender_spd_transformed_classifier,
                        marital_di_transformed_classifier, marital_spd_transformed_classifier],
    'Change Direction': [
        classify_change(gender_di_transformed_data, gender_di_transformed_classifier, 'DI'),
        classify_change(gender_spd_transformed_data, gender_spd_transformed_classifier, 'SPD'),
        classify_change(marital_di_transformed_data, marital_di_transformed_classifier, 'DI'),
        classify_change(marital_spd_transformed_data, marital_spd_transformed_classifier, 'SPD')
    ]
}

# Make table
df_classifier_trans = pd.DataFrame(classifier_transform_changes)
df_classifier_trans['Absolute Change'] = abs(df_classifier_trans['Classifier Value'] - 
                                             df_classifier_trans['Data Value'])
print(df_classifier_trans.to_string(index=False))

# TABLE 3: Compare original model vs transformed model
print("\n" + "="*80)
print("TABLE 3: OVERALL COMPARISON - Original Model vs Transformed Model")
print("-" * 80)

# Compare the two models
overall_comparison = {
    'Protected Class': ['Gender', 'Gender', 'MaritalStatus', 'MaritalStatus'],
    'Fairness Metric': ['Disparate Impact', 'Statistical Parity Diff', 'Disparate Impact', 'Statistical Parity Diff'],
    'Original Classifier': [gender_di_original_classifier, gender_spd_original_classifier,
                           marital_di_original_classifier, marital_spd_original_classifier],
    'Transformed Classifier': [gender_di_transformed_classifier, gender_spd_transformed_classifier,
                              marital_di_transformed_classifier, marital_spd_transformed_classifier],
    'Change Direction': [
        classify_change(gender_di_original_classifier, gender_di_transformed_classifier, 'DI'),
        classify_change(gender_spd_original_classifier, gender_spd_transformed_classifier, 'SPD'),
        classify_change(marital_di_original_classifier, marital_di_transformed_classifier, 'DI'),
        classify_change(marital_spd_original_classifier, marital_spd_transformed_classifier, 'SPD')
    ]
}

# Make table
df_overall = pd.DataFrame(overall_comparison)
df_overall['Absolute Change'] = abs(df_overall['Transformed Classifier'] - 
                                    df_overall['Original Classifier'])
print(df_overall.to_string(index=False))

# Summary of what we found
print("\n" + "="*80)
print("KEY FINDINGS:")
print("="*80)
print("✓ Reweighing improved fairness for MaritalStatus (SPD improved from -0.067 to -0.041)")
print("✓ Gender fairness remained stable across all stages (all SPD values within fair region)")
print("✓ Transformed classifier showed best fairness for MaritalStatus (SPD = -0.021)")
print("⚠ Gender SPD slightly worsened after transformation (0.022 → 0.038)")
print("="*80)

## 5.3: Team Member Individual Responses

### Team Member 1: Mark P Abbott (mabbott7)

**Q1: Did any of these approaches seem to work to mitigate bias (or increase fairness)? Explain your reasoning.**

[Your response here - at least one paragraph]

**Q2: Did any group receive a positive advantage?**

[Your response here - at least one paragraph]

**Q3: Were any group disadvantaged by these approaches?**

[Your response here - at least one paragraph]

**Q4: What issues would arise if you used these methods to mitigate bias?**

[Your response here - at least one paragraph]

---

### Team Member 2: Michael Countouris (mcountouris3)

**Q1: Did any of these approaches seem to work to mitigate bias (or increase fairness)? Explain your reasoning.**

[Your response here - at least one paragraph]

**Q2: Did any group receive a positive advantage?**

[Your response here - at least one paragraph]

**Q3: Were any group disadvantaged by these approaches?**

[Your response here - at least one paragraph]

**Q4: What issues would arise if you used these methods to mitigate bias?**

[Your response here - at least one paragraph]

---

### Team Member 3: Soon Ryu (sryu71)

**Q1: Did any of these approaches seem to work to mitigate bias (or increase fairness)? Explain your reasoning.**

[Your response here - at least one paragraph]

**Q2: Did any group receive a positive advantage?**

[Your response here - at least one paragraph]

**Q3: Were any group disadvantaged by these approaches?**

[Your response here - at least one paragraph]

**Q4: What issues would arise if you used these methods to mitigate bias?**

[Your response here - at least one paragraph]

## 5.2: Analysis - Which Fairness Metric is Best?

### Fairness Metric Evaluation

**Statistical Parity Difference (SPD) is the better fairness metric for this analysis.**

### Justification:

1. **Interpretability**: SPD directly measures the difference in favorable outcome rates between privileged and unprivileged groups. A value of 0 indicates perfect fairness, and values within [-0.1, 0.1] are generally considered acceptable. This makes SPD more intuitive to interpret than Disparate Impact.

2. **Sensitivity to Changes**: SPD shows clearer trends across the different stages of our analysis:
   - For Gender: SPD increased from 0.022 (original data) to 0.038 (transformed data/classifier), indicating reweighing may have slightly worsened fairness
   - For MaritalStatus: SPD improved from -0.067 (original data) to -0.041 (transformed data) to -0.021 (transformed classifier), showing positive bias mitigation

3. **Fair Region Alignment**: 
   - For Gender, all SPD values remain within the fair region (-0.1 to 0.1)
   - For MaritalStatus, SPD values moved closer to 0 (ideal fairness) after transformation and classification

4. **Practical Application**: SPD directly translates to real-world impact - it tells us the percentage point difference in attrition rates between groups, which is meaningful for HR decision-making.

### Key Observations:

- **Disparate Impact** values remained relatively stable across stages, all within the fair region (0.8-1.25)
- **Statistical Parity Difference** showed more nuanced changes, revealing that reweighing was more effective for MaritalStatus than Gender
- The transformed classifier showed the best fairness for MaritalStatus (SPD = -0.021), closest to ideal fairness